# State Farm Distracted Driver Detection - VGG16 Continued

This notebook contains the final attempt at the Kaggle State Farm Distracted Driver Detection competition. The purpose is to train the best possible model, as well as testing the external vgg16, utils and plot libraries.

## Initial Setup

Import libraries and functions for future use.

In [1]:
# Plots displayed inline in notebook
%matplotlib inline

# Make Python 3 consistent
from __future__ import print_function, division

# Make help libraries available
import sys

sys.path.append('/home/ubuntu/personal-libraries')

In [64]:
import numpy as np
import pandas as pd
import gc

from kerastools.vgg16 import Vgg16
from kerastools.utils import get_batches, save_array, load_array, get_classes, do_clip

from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, Flatten, Dropout
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image

## Define model

We setup our initial VGG16 model

In [3]:
vgg = Vgg16()
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
__________

## Setup batches

We define out validation and training badges for modelling

In [4]:
batch_size = 32

#path = ''
path = 'sample/'

train_batches = vgg.get_batches(path + 'train', batch_size = batch_size)
val_batches = vgg.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 1000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


## Finetune model - Sample

We need to adjust the standard VGG model to our new input with 10 classes, so we finetune it.

In [5]:
vgg.finetune(train_batches)
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
__________

We train the model using the default learning rate of 0.001 for a single epoch

In [6]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
31/31 [==============================] - 26s - loss: 4.0402 - acc: 0.1583 - val_loss: 2.2574 - val_acc: 0.2396


We see that the accuracy increases fine on the sample, so we increase the learning rate.

In [8]:
vgg.model.optimizer.lr = 0.1

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 25s - loss: 2.8166 - acc: 0.2632 - val_loss: 1.8976 - val_acc: 0.3971
Epoch 2/4
31/31 [==============================] - 25s - loss: 2.2921 - acc: 0.3780 - val_loss: 1.6727 - val_acc: 0.3824
Epoch 3/4
31/31 [==============================] - 25s - loss: 2.0350 - acc: 0.4175 - val_loss: 1.4732 - val_acc: 0.4559
Epoch 4/4
31/31 [==============================] - 25s - loss: 1.8398 - acc: 0.4750 - val_loss: 1.2862 - val_acc: 0.6029


It actually seems to be generalising nicely. We try 4 more epochs.

In [9]:
vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 26s - loss: 1.5728 - acc: 0.5443 - val_loss: 1.4228 - val_acc: 0.4375
Epoch 2/4
31/31 [==============================] - 25s - loss: 1.6685 - acc: 0.5161 - val_loss: 1.8084 - val_acc: 0.3824
Epoch 3/4
31/31 [==============================] - 25s - loss: 1.6304 - acc: 0.5545 - val_loss: 1.6064 - val_acc: 0.4412
Epoch 4/4
31/31 [==============================] - 25s - loss: 1.2680 - acc: 0.6119 - val_loss: 1.0764 - val_acc: 0.6029


Damn. Our valuation accuracy is destroyed. Try 4 more epochs with lower learning rate.

In [10]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 26s - loss: 1.3070 - acc: 0.6047 - val_loss: 1.2782 - val_acc: 0.5521
Epoch 2/4
31/31 [==============================] - 25s - loss: 1.1501 - acc: 0.6269 - val_loss: 1.1022 - val_acc: 0.6618
Epoch 3/4
31/31 [==============================] - 25s - loss: 1.1370 - acc: 0.6601 - val_loss: 1.6280 - val_acc: 0.4412
Epoch 4/4
31/31 [==============================] - 25s - loss: 1.2141 - acc: 0.6311 - val_loss: 0.9508 - val_acc: 0.6618


Seems, this is as far as we can get on the sample data set. A pretty good base line in the area of 0.5

## Finetune model - Full data

We continue our finetuning on the full data set.

In [13]:
path = ''

train_batches = vgg.get_batches(path + 'train', batch_size = batch_size)
val_batches = vgg.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.


We start with a single epoch

In [14]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
613/613 [==============================] - 548s - loss: 1.2254 - acc: 0.6352 - val_loss: 1.4150 - val_acc: 0.5686


We increase the learning rate and see, where that takes us.

In [16]:
vgg.model.optimizer.lr = 0.1

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 544s - loss: 1.0154 - acc: 0.6959 - val_loss: 1.2824 - val_acc: 0.6004
Epoch 2/4
613/613 [==============================] - 544s - loss: 0.9906 - acc: 0.7109 - val_loss: 1.3605 - val_acc: 0.5737
Epoch 3/4
613/613 [==============================] - 545s - loss: 0.9601 - acc: 0.7237 - val_loss: 1.1499 - val_acc: 0.6413
Epoch 4/4
613/613 [==============================] - 545s - loss: 0.9860 - acc: 0.7249 - val_loss: 1.1938 - val_acc: 0.6301


And then we lower the learning rate again, and see where we end up.

In [17]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 546s - loss: 0.9762 - acc: 0.7356 - val_loss: 1.4268 - val_acc: 0.5943
Epoch 2/4
613/613 [==============================] - 545s - loss: 0.9961 - acc: 0.7316 - val_loss: 1.2792 - val_acc: 0.6441
Epoch 3/4
613/613 [==============================] - 546s - loss: 0.9964 - acc: 0.7431 - val_loss: 1.3723 - val_acc: 0.6004
Epoch 4/4
613/613 [==============================] - 547s - loss: 0.9599 - acc: 0.7479 - val_loss: 1.7435 - val_acc: 0.5441


The valuation accuracy is very fluctuating, and generally we are overfitting. Lets try and make more layers trainable and see, if that helps things along.

In [19]:
layers = vgg.model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index, layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable = True

And then we rerun the training. First one epoch with low learning rate.

In [21]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
613/613 [==============================] - 545s - loss: 0.9931 - acc: 0.7492 - val_loss: 1.1719 - val_acc: 0.6535


Then four epochs with a higher learning rate.

In [22]:
vgg.model.optimizer.lr = 0.1

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 547s - loss: 1.0058 - acc: 0.7514 - val_loss: 1.4089 - val_acc: 0.5968
Epoch 2/4
613/613 [==============================] - 546s - loss: 0.9926 - acc: 0.7521 - val_loss: 1.9295 - val_acc: 0.5303
Epoch 3/4
613/613 [==============================] - 546s - loss: 1.0180 - acc: 0.7499 - val_loss: 1.5204 - val_acc: 0.6160
Epoch 4/4
613/613 [==============================] - 546s - loss: 1.0084 - acc: 0.7563 - val_loss: 1.2607 - val_acc: 0.6694


And then four epochs with a lower learning rate again.

In [23]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 546s - loss: 1.0034 - acc: 0.7575 - val_loss: 1.2772 - val_acc: 0.6694
Epoch 2/4
613/613 [==============================] - 545s - loss: 1.0078 - acc: 0.7573 - val_loss: 1.5594 - val_acc: 0.5842
Epoch 3/4
613/613 [==============================] - 545s - loss: 1.0057 - acc: 0.7608 - val_loss: 1.3791 - val_acc: 0.6199
Epoch 4/4
613/613 [==============================] - 545s - loss: 1.0109 - acc: 0.7602 - val_loss: 1.4240 - val_acc: 0.6311


That did little to improve things. Let's try an even lower learning rate.

In [24]:
vgg.model.optimizer.lr = 0.00001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 546s - loss: 1.0160 - acc: 0.7593 - val_loss: 1.6110 - val_acc: 0.6062
Epoch 2/4
613/613 [==============================] - 545s - loss: 1.0237 - acc: 0.7632 - val_loss: 1.1922 - val_acc: 0.6734
Epoch 3/4
613/613 [==============================] - 546s - loss: 1.0405 - acc: 0.7630 - val_loss: 1.6781 - val_acc: 0.5798
Epoch 4/4
613/613 [==============================] - 546s - loss: 1.0344 - acc: 0.7638 - val_loss: 1.5770 - val_acc: 0.6315


We're not improving at all. Let's save the weights and try a different approach.

In [25]:
vgg.model.save_weights('models/base_vgg16.h5')

## Improved VGG

We continue using the VGG16 network, but attempt to improve it. That is we want to keep the pretrained convolutional layers fixed, but use a new architecture for the dense layers.

We start by defining a new VGG16() model.

In [3]:
vgg = Vgg16()
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
__________

We then proceed to find the last max pooling layer of the model.

In [4]:
# Define convolutional layers
last_conv_idx = [i for i, l in enumerate(vgg.model.layers) if type(l) is MaxPooling2D][-1]
conv_layers = vgg.model.layers[:last_conv_idx + 1]

We can then define a model using only the convolutional layers.

In [5]:
conv_model = Sequential(conv_layers)
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
__________

The idea is now, that we want to pre-computer all of our data through the convolutional layers. This will drastically reduce the training time, once we start experimenting with dense model architecture.

We start by defining our batches.

In [6]:
path = ''

train_batches = get_batches(path + 'train', batch_size = 44, target_size = (224, 224), shuffle = False)

valid_batches = get_batches(path + 'valid', batch_size = 50, target_size = (224, 224), shuffle = False)

test_batches = get_batches(path + 'test', batch_size = 2, target_size = (224, 224), shuffle = False, class_mode = None)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


We also extract labels and classes for each dataset.

In [7]:
(val_classes, trn_classes, val_labels, trn_labels, 
 val_filenames, filenames, test_filenames) = get_classes(path)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


We then pre-compute each of our datasets and save the numpy arrays. This eats a lot of memory on the poor AWS instance, so after each data computation, we do some cleanup to realease the memory. We save and load using bcolz, as it utilises great compression and does I/O very fast.

In [ ]:
conv_feat = conv_model.predict_generator(train_batches, np.int(train_batches.samples / train_batches.batch_size))
save_array(path + 'results/conv_computed/conv_feat.dat', conv_feat)

del conv_feat
gc.collect()

In [20]:
conv_val_feat = conv_model.predict_generator(valid_batches, np.int(valid_batches.samples / valid_batches.batch_size))
save_array(path + 'results/conv_computed/conv_val_feat.dat', conv_val_feat)

del conv_val_feat
gc.collect()

0

In [21]:
conv_test_feat = conv_model.predict_generator(test_batches, np.int(test_batches.samples / test_batches.batch_size))
save_array(path + 'results/conv_computed/conv_test_feat.dat', conv_test_feat)

del conv_test_feat
gc.collect()

0

And finally, we can load the three feature sets

In [8]:
conv_feat = load_array('results/conv_computed/conv_feat.dat')
conv_val_feat = load_array('results/conv_computed/conv_val_feat.dat')
conv_test_feat = load_array('results/conv_computed/conv_test_feat.dat')

## VGG model with batchnorm dense laysers on pretrained conv layers

Since we've pre-computed the output of the last convolutional layer, we need to create a network that takes that as input, and predicts our 10 classes. Let's try using a simplified version of VGG's dense layers.

In [9]:
def batch_norm_model(base_model, p):
    """
    Function providing a more modern dense layer to the VGG16.
    I.e. it utilizes batch normalisation.
    
    Args:
        - base_model: A keras model object.
        - p: Total level of dropout (will be devided by two in first layers)
        
    Returns:
        - A keras model object ready to be compiled.
    """
    
    inputs = Input(shape = base_model[-1].output_shape[1:])
        
    flat_layer = Flatten()(inputs)
        
    dense_layer_1 = Dropout(p / 2)(flat_layer)
    dense_layer_1 = Dense(128, activation='relu')(dense_layer_1)
    dense_layer_2 = BatchNormalization()(dense_layer_1)
    dense_layer_2 = Dropout(p / 2)(dense_layer_2)
    dense_layer_2 = Dense(128, activation='relu')(dense_layer_2)
    dense_layer_3 = BatchNormalization()(dense_layer_2)
    dense_layer_3 = Dropout(p)(dense_layer_3)
    dense_layer_3 = Dense(10, activation='softmax')(dense_layer_3)
    
    model = Model(inputs = inputs, outputs = dense_layer_3)
    
    return model

# Set level of dropout
p = 0.8

# Set batch size
batch_size = 32

Lets define and compile the model. We run a single epoch on the default learning rate.

In [14]:
bn_model = batch_norm_model(conv_layers, p)

bn_model.compile(optimizer = Adam(lr = 0.001),
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])

bn_model.fit(x = conv_feat,
             y = trn_labels,
             batch_size = batch_size,
             epochs = 1,
             validation_data = (conv_val_feat, val_labels))

Train on 19624 samples, validate on 2800 samples
Epoch 1/1
19624/19624 [==============================] - 8s - loss: 1.4120 - acc: 0.6059 - val_loss: 0.6459 - val_acc: 0.8146


So very fast(!), we see a major improvement over the base VGG16 model. Lets continue running for a few more epochs at a lower learning rate.

In [15]:
bn_model.optimizer.lr = 0.0001

bn_model.fit(x = conv_feat,
             y = trn_labels,
             batch_size = batch_size,
             epochs = 4,
             validation_data = (conv_val_feat, val_labels))

Train on 19624 samples, validate on 2800 samples
Epoch 1/4
19624/19624 [==============================] - 8s - loss: 0.2527 - acc: 0.9235 - val_loss: 0.5918 - val_acc: 0.7843
Epoch 2/4
19624/19624 [==============================] - 8s - loss: 0.1348 - acc: 0.9615 - val_loss: 0.5954 - val_acc: 0.7939
Epoch 3/4
19624/19624 [==============================] - 8s - loss: 0.0932 - acc: 0.9732 - val_loss: 0.7723 - val_acc: 0.7707
Epoch 4/4
19624/19624 [==============================] - 8s - loss: 0.0766 - acc: 0.9784 - val_loss: 0.6379 - val_acc: 0.8000


So we are starting to overfit quite a bit. We have pretty much done as much as we could with the base model, in terms of regularisation. We have added dropout and batchnorm. Time to take a look and see if we can improve the input to the model.

## VGG model with precomputed augmentation and dropout.

We precompute some augmented data in order to reduce the overfitting of out model. We start by setting the level of preprocessing and then define a new bunch of batches.

In [10]:
gen_t = image.ImageDataGenerator(rotation_range = 15,
                                 height_shift_range = 0.05,
                                 shear_range = 0.1,
                                 channel_shift_range = 20,
                                 width_shift_range = 0.1)

da_batches = get_batches(path + 'train',
                         gen_t,
                         batch_size = 44,
                         shuffle = False,
                         target_size = (224, 224))

Found 19624 images belonging to 10 classes.


We then make 5 full cycles (epochs) of the training data, precomputing the convolutional layer on the augmented images.

In [18]:
da_conv_feat = conv_model.predict_generator(da_batches, np.int(da_batches.samples / da_batches.batch_size) * 5)
save_array(path + 'results/conv_computed/da_conv_feat.dat', da_conv_feat)

del da_conv_feat
gc.collect()

359

We can then load the pre-compuited and augmented data.

In [11]:
da_conv_feat = load_array(path + 'results/conv_computed/da_conv_feat.dat')

And then concatenate it with our existing precomputed training data.

In [12]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

We now have 6 times as many features, but luckily order is preserved, so we can just multiply the labels with 6.

In [14]:
da_trn_labels = np.concatenate([trn_labels] * 6)

We can then run the model again, but we try use use bigger dense layers and some more dropout.

In [13]:
def batch_norm_model_bigger(base_model, p):
    
    inputs = Input(shape = base_model[-1].output_shape[1:])
        
    flat_layer = Flatten()(inputs)
        
    dense_layer_1 = Dropout(p)(flat_layer)
    dense_layer_1 = Dense(256, activation='relu')(dense_layer_1)
    dense_layer_2 = BatchNormalization()(dense_layer_1)
    dense_layer_2 = Dropout(p)(dense_layer_2)
    dense_layer_2 = Dense(256, activation='relu')(dense_layer_2)
    dense_layer_3 = BatchNormalization()(dense_layer_2)
    dense_layer_3 = Dropout(p)(dense_layer_3)
    dense_layer_3 = Dense(10, activation='softmax')(dense_layer_3)
    
    model = Model(inputs = inputs, outputs = dense_layer_3)
    
    return model

# Define level of dropout
p = 0.8

# Define batch size
batch_size = 32

Re then compile the model, and run it for a single epoch on the augmented data at the defulat learning rate.

In [17]:
bn_model_bigger = batch_norm_model_bigger(conv_layers, p)

bn_model_bigger.compile(optimizer = Adam(lr = 0.001),
                        loss = 'categorical_crossentropy',
                        metrics = ['accuracy'])

bn_model_bigger.fit(x = da_conv_feat,
                    y = da_trn_labels,
                    batch_size = batch_size,
                    epochs = 1,
                    validation_data = (conv_val_feat, val_labels))

Train on 117744 samples, validate on 2800 samples
Epoch 1/1
117744/117744 [==============================] - 50s - loss: 1.4990 - acc: 0.5158 - val_loss: 0.5592 - val_acc: 0.8286


So this model is a bit heavier. Already we are underfitting quite a bit. We increase the learning rate and run for 4 epochs.

In [19]:
bn_model_bigger.optimizer.lr = 0.01

bn_model_bigger.fit(x = da_conv_feat,
                    y = da_trn_labels,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 117744 samples, validate on 2800 samples
Epoch 1/4
117744/117744 [==============================] - 49s - loss: 0.7133 - acc: 0.7639 - val_loss: 0.4834 - val_acc: 0.8354
Epoch 2/4
117744/117744 [==============================] - 48s - loss: 0.5696 - acc: 0.8162 - val_loss: 0.4852 - val_acc: 0.8204
Epoch 3/4
117744/117744 [==============================] - 48s - loss: 0.4978 - acc: 0.8402 - val_loss: 0.4416 - val_acc: 0.8496
Epoch 4/4
117744/117744 [==============================] - 48s - loss: 0.4542 - acc: 0.8551 - val_loss: 0.4460 - val_acc: 0.8571


Now our training and valuation accuracy seems more ballanced. Let's lower the learning rate again and train for 4 more epochs.

In [20]:
bn_model_bigger.optimizer.lr = 0.0001

bn_model_bigger.fit(x = da_conv_feat,
                    y = da_trn_labels,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 117744 samples, validate on 2800 samples
Epoch 1/4
117744/117744 [==============================] - 48s - loss: 0.4255 - acc: 0.8660 - val_loss: 0.4687 - val_acc: 0.8404
Epoch 2/4
117744/117744 [==============================] - 48s - loss: 0.4019 - acc: 0.8731 - val_loss: 0.5088 - val_acc: 0.8336
Epoch 3/4
117744/117744 [==============================] - 48s - loss: 0.3789 - acc: 0.8804 - val_loss: 0.4887 - val_acc: 0.8454
Epoch 4/4
117744/117744 [==============================] - 48s - loss: 0.3623 - acc: 0.8860 - val_loss: 0.4748 - val_acc: 0.8339


Seems like we are levelling off at this point. Let's attempt a final improvement by using pseudo labelling.

## Pseudolabeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. Afterwards we add the test set as well.

In [22]:
val_pseudo = bn_model_bigger.predict(conv_val_feat, batch_size = 50)

We concatenate thse pseudo labels with our training labels

In [24]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

And train our model using the extended data set.

In [26]:
bn_model_bigger.optimizer.lr = 0.001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 1,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/1
120544/120544 [==============================] - 49s - loss: 0.3544 - acc: 0.8902 - val_loss: 0.4987 - val_acc: 0.8382


We do not really see much of an improvement. Let's try 4 more epochs.

In [27]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/4
120544/120544 [==============================] - 50s - loss: 0.3426 - acc: 0.8951 - val_loss: 0.4796 - val_acc: 0.8421
Epoch 2/4
120544/120544 [==============================] - 50s - loss: 0.3360 - acc: 0.8967 - val_loss: 0.5171 - val_acc: 0.8375
Epoch 3/4
120544/120544 [==============================] - 50s - loss: 0.3269 - acc: 0.8995 - val_loss: 0.5132 - val_acc: 0.8371
Epoch 4/4
120544/120544 [==============================] - 50s - loss: 0.3224 - acc: 0.9010 - val_loss: 0.5321 - val_acc: 0.8364


Now we are crossing the 0.9 threshold of accuracy. Lets lower the learning rate and train for 4 more epochs and see where that gets us.

In [29]:
bn_model_bigger.optimizer.lr = 0.00001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/4
120544/120544 [==============================] - 50s - loss: 0.3134 - acc: 0.9038 - val_loss: 0.5191 - val_acc: 0.8411
Epoch 2/4
120544/120544 [==============================] - 50s - loss: 0.3062 - acc: 0.9060 - val_loss: 0.5285 - val_acc: 0.8414
Epoch 3/4
120544/120544 [==============================] - 50s - loss: 0.3033 - acc: 0.9071 - val_loss: 0.4933 - val_acc: 0.8443
Epoch 4/4
120544/120544 [==============================] - 50s - loss: 0.2933 - acc: 0.9103 - val_loss: 0.4806 - val_acc: 0.8475


So we do see a pretty nice improvement. Enough to warrent us trying with the entire test set.

In [30]:
test_pseudo = bn_model_bigger.predict(conv_test_feat, batch_size = 2)

We concatenate thse pseudo labels with our training and valuation pseudo labels

In [31]:
comb_pseudo = np.concatenate([comb_pseudo, test_pseudo])
comb_feat = np.concatenate([comb_feat, conv_test_feat])

And train our model using the extended data set.

In [32]:
bn_model_bigger.optimizer.lr = 0.001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 1,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/1
200270/200270 [==============================] - 84s - loss: 0.5545 - acc: 0.8444 - val_loss: 0.4587 - val_acc: 0.8454


Hrm, too early to say, but valuation accuracy holds still. Let's run 4 more epochs.

In [33]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/4
200270/200270 [==============================] - 84s - loss: 0.5158 - acc: 0.8610 - val_loss: 0.4763 - val_acc: 0.8346
Epoch 2/4
200270/200270 [==============================] - 83s - loss: 0.4985 - acc: 0.8692 - val_loss: 0.4769 - val_acc: 0.8339
Epoch 3/4
200270/200270 [==============================] - 83s - loss: 0.4936 - acc: 0.8704 - val_loss: 0.4463 - val_acc: 0.8493
Epoch 4/4
200270/200270 [==============================] - 84s - loss: 0.4850 - acc: 0.8746 - val_loss: 0.4865 - val_acc: 0.8396


Let's lower the learning rate again and run 4 more epochs.

In [34]:
bn_model_bigger.optimizer.lr = 0.00001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/4
200270/200270 [==============================] - 83s - loss: 0.4803 - acc: 0.8757 - val_loss: 0.4721 - val_acc: 0.8389
Epoch 2/4
200270/200270 [==============================] - 84s - loss: 0.4764 - acc: 0.8771 - val_loss: 0.4542 - val_acc: 0.8464
Epoch 3/4
200270/200270 [==============================] - 83s - loss: 0.4750 - acc: 0.8788 - val_loss: 0.4759 - val_acc: 0.8396
Epoch 4/4
200270/200270 [==============================] - 83s - loss: 0.4700 - acc: 0.8799 - val_loss: 0.4844 - val_acc: 0.8364


We are improving slowly. Let's run for 10 more epochs.

In [36]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 10,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/10
200270/200270 [==============================] - 83s - loss: 0.4671 - acc: 0.8815 - val_loss: 0.4400 - val_acc: 0.8461
Epoch 2/10
200270/200270 [==============================] - 83s - loss: 0.4642 - acc: 0.8820 - val_loss: 0.5003 - val_acc: 0.8371
Epoch 3/10
200270/200270 [==============================] - 83s - loss: 0.4617 - acc: 0.8831 - val_loss: 0.5024 - val_acc: 0.8386
Epoch 4/10
200270/200270 [==============================] - 84s - loss: 0.4578 - acc: 0.8849 - val_loss: 0.5042 - val_acc: 0.8339
Epoch 5/10
200270/200270 [==============================] - 83s - loss: 0.4557 - acc: 0.8858 - val_loss: 0.4949 - val_acc: 0.8368
Epoch 6/10
200270/200270 [==============================] - 83s - loss: 0.4552 - acc: 0.8843 - val_loss: 0.4806 - val_acc: 0.8379
Epoch 7/10
200270/200270 [==============================] - 83s - loss: 0.4532 - acc: 0.8865 - val_loss: 0.4609 - val_acc: 0.8436
Epoch 8/10
200270/200270 [==============

Having trained the model, we save the weights.

In [37]:
bn_model_bigger.save_weights('models/batchnorm_vgg16.h5')

## Submitting to Kaggle

We finally submit the improved model to Kaggle.

We start by finding the optimal level of clipping.

In [48]:
valid_batches_pred = get_batches(path + 'valid', batch_size = 50, target_size = (224, 224), shuffle = False, class_mode = None)
conv_val_feat_pred = conv_model.predict_generator(valid_batches_pred, np.int(valid_batches_pred.samples / valid_batches_pred.batch_size))

val_predictions = bn_model_bigger.predict(conv_val_feat_pred, batch_size = 50)

Found 2800 images belonging to 10 classes.


In [57]:
def do_clip(arr, mx): return np.clip(arr, (1 - mx) / 9, mx)

We then proceed to determine the optimal level of clipping using the validation data set.

In [58]:
test_clip = []
for i in np.arange(0.70, 1.0, 0.01):
    test_clip.append([i, categorical_crossentropy(val_labels, do_clip(val_predictions, i)).eval().mean()])

min(test_clip, key = lambda x: x[1])

[1.0000000000000002, 0.48164355679327142]

Here it is said, that no clipping is best. Weird. Lets submit two. One clipped and one not clipped. First we compute the predictions.

In [59]:
test_predictions = bn_model_bigger.predict(conv_test_feat, batch_size = 2)

Define classes

In [61]:
classes = sorted(valid_batches.class_indices, key = valid_batches.class_indices.get)

Then we make a cliping based on earlier experience

In [62]:
sumbit_pred = do_clip(test_predictions, 0.89)

Then we prepare a submission without clipping.

In [65]:
submission_no_clip = pd.DataFrame(test_predictions, columns = classes)
submission_no_clip.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission_no_clip.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.046099,0.006723,0.004307,0.000682,0.001594,0.001567,0.026646,0.020867,0.016716,0.874798
1,img_14887.jpg,0.689640,0.007839,0.000621,0.002267,0.001608,0.003189,0.001825,0.000393,0.005292,0.287327
2,img_62885.jpg,0.005301,0.000154,0.000257,0.017661,0.973055,0.000493,0.000514,0.000016,0.000978,0.001571
3,img_45125.jpg,0.002003,0.007311,0.017353,0.000462,0.002328,0.000509,0.656182,0.006239,0.302687,0.004926
4,img_22633.jpg,0.138601,0.055678,0.011123,0.001665,0.004929,0.012565,0.022692,0.007558,0.186375,0.558814


And a submission with clipping

In [66]:
submission_clip = pd.DataFrame(sumbit_pred, columns = classes)
submission_clip.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission_clip.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.046099,0.012222,0.012222,0.012222,0.012222,0.012222,0.026646,0.020867,0.016716,0.874798
1,img_14887.jpg,0.689640,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.287327
2,img_62885.jpg,0.012222,0.012222,0.012222,0.017661,0.890000,0.012222,0.012222,0.012222,0.012222,0.012222
3,img_45125.jpg,0.012222,0.012222,0.017353,0.012222,0.012222,0.012222,0.656182,0.012222,0.302687,0.012222
4,img_22633.jpg,0.138601,0.055678,0.012222,0.012222,0.012222,0.012565,0.022692,0.012222,0.186375,0.558814


Finally we save the two submissions

In [67]:
submission_file_name_no_clip = 'results/augmented-pseudo-vgg-no-clip.gz'
submission_no_clip.to_csv(submission_file_name_no_clip, index = False, compression = 'gzip')

submission_file_name_clip = 'results/augmented-pseudo-vgg-clip.gz'
submission_clip.to_csv(submission_file_name_clip, index = False, compression = 'gzip')

In [70]:
from IPython.display import FileLink
FileLink('results/augmented-pseudo-vgg-no-clip.gz')

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/results/augmented-pseudo-vgg-no-clip.gz

In [71]:
FileLink('results/augmented-pseudo-vgg-clip.gz')

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/results/augmented-pseudo-vgg-clip.gz

Turns out that in this case, the no clipping submission actually performed best, by a absolute 0.03.